# Clinical Text Classification

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
## path to the data
train_path = '/content/gdrive/My Drive/mtsamples.csv'

In [ ]:
# install required libraries
!pip install nlpaug

In [ ]:
## Checking the gpu device
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

#plot
import matplotlib.pyplot as plt
import seaborn as sns

import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action

## Data loading and pre-processing

In [ ]:
medical_data = pd.read_csv(train_path, index_col=0)

In [ ]:
len(medical_data)

In [ ]:
## getting only the required data from the dataframe
medical_data = medical_data[['transcription','medical_specialty']]
medical_data.dropna(inplace=True)

In [ ]:
len(medical_data)

In [ ]:
## removing duplicate transcriptions
medical_data = medical_data.drop_duplicates(subset=['transcription'], keep='first')

In [ ]:
len(medical_data)

In [ ]:
## removing notes
notes_list = [' Letters', ' SOAP / Chart / Progress Notes', ' Emergency Room Reports',
              ' Discharge Summary', ' Consult - History and Phy.', ' Office Notes']
medical_data = medical_data[~medical_data['medical_specialty'].isin(notes_list)]

In [ ]:
len(medical_data)

In [ ]:
## list of medical specialties present in the data
medical_data['medical_specialty'].unique()

In [ ]:
## count for each specialty
medical_data.groupby('medical_specialty').count()['transcription'].reset_index()

In [ ]:
#medical_data = medical_data.groupby('medical_specialty').filter(lambda x : len(x)>20)

In [ ]:
## merging ' Cosmetic / Plastic Surgery' and ' Dermatology' into one category
medical_data = medical_data.replace(' Cosmetic / Plastic Surgery',' Dermatology')
label_count = medical_data.groupby('medical_specialty').count()['transcription'].reset_index()
label_count

In [ ]:
## bar plot for the number of cases for each specialty
plt.figure(figsize=(20,8))
x = medical_data.medical_specialty.value_counts()
sns.barplot(x=x.index,y=x)
#plt.tight_layout()
plt.xticks(
    rotation=45,
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large'
)
plt.gca().set_ylabel('samples')

## Data Augmentation

In [ ]:
## data augmentation to increase the number of samples to balance the data
aug = naw.SynonymAug(aug_src='wordnet',aug_max= 24)
for i in range(0, len(label_count)-1):
    aug_val = int(round(label_count.transcription.max() / label_count.transcription[i],0)) - 1
    filtered_data = medical_data.loc[medical_data['medical_specialty'] == label_count.medical_specialty[i]]
    for j in filtered_data.index:
        temps=aug.augment(filtered_data.transcription[j],n=aug_val)
        for sent in temps:
            medical_data1 = {'transcription': sent, 'medical_specialty': label_count.medical_specialty[i]}
            medical_data = medical_data.append(medical_data1, ignore_index = True)
    #print(medical_data.shape)

In [ ]:
medical_data.groupby('medical_specialty').count()['transcription'].reset_index()

In [ ]:
plt.figure(figsize=(20,8))
x = medical_data.medical_specialty.value_counts()
sns.barplot(x=x.index,y=x)
#plt.tight_layout()
plt.xticks(
    rotation=45,
    horizontalalignment='right',
    fontweight='light',
    fontsize='x-large'
)
plt.gca().set_ylabel('samples')

## Text pre-processing

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import regex as re
import string

from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

In [ ]:
def text_preprocessing(data):
    data = str(data).lower()
    data = re.sub('\[.*?\]', '', data)
    data = re.sub('https?://\S+|www\.\S+', '', data)
    data = re.sub('<.*?>+', '', data)
    data = re.sub('[%s]' % re.escape(string.punctuation), '', data)
    data = re.sub('\n', '', data)
    data = re.sub('\w*\d\w*', '', data)
    return data

def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in STOPWORDS])
medical_data["transcription"] = medical_data["transcription"].apply(lambda text: text_preprocessing(text))
medical_data["transcription"] = medical_data["transcription"].apply(lambda text: remove_stopwords(text))
medical_data['tr_word_count']= medical_data['transcription'].apply(lambda x: len(str(x).split(' ')))

In [ ]:
Y = pd.get_dummies(medical_data['medical_specialty']).values
print('Shape of label tensor:', Y.shape)

In [ ]:
medical_data.hist(column = 'tr_word_count')

In [ ]:
documents = []

for doc in medical_data['transcription']:
    documents.append(doc.split(' '))

## Word2Vec model

In [ ]:
from gensim.models import Word2Vec
# Training the Word2Vec model
model = Word2Vec(documents, #Word list
                   min_count=5, #Ignore all words with total frequency lower than this
                   workers=4, #Number of CPUs
                   vector_size=100,  #Embedding size
                   window=5 #Maximum Distance between current and predicted word
                  )

In [ ]:
#How many words in the model and how many features
model.wv.vectors.shape

In [ ]:
all_words = model.wv.index_to_key               #key_to_index
top_words = model.wv.index_to_key[0:5]
top_words

In [ ]:
print('Vocabulary size: %d' % len(all_words))

In [ ]:
# Most similar words
similar = model.wv.most_similar('anesthetic')
similar

In [ ]:
model.save("word2vec_Medical Specialty Classification6")

## Data split

In [ ]:
from sklearn.model_selection import train_test_split
X = medical_data['transcription']
y = Y
#train test validation split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify = y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1, stratify = y_train)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
# building tokenizer to form a token for each word in each transcription

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding

In [ ]:
#Convert words to integers

top_words = 10000 #reducing no of words to increase the processing speed

tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_val = tokenizer.texts_to_sequences(X_val)

In [ ]:
# pad sequenc to make each transcription of the same size
#taking 300  based on avg length of the transcription form histogram
padding_max_len = 300

X_train = pad_sequences(X_train, maxlen=padding_max_len, padding='post')
X_test = pad_sequences(X_test, maxlen=padding_max_len, padding='post')
X_val = pad_sequences(X_val, maxlen=padding_max_len, padding='post')

In [ ]:
# Building embedding matrix from above Word2Vec model(pre trained model)

word2vec_model = Word2Vec.load("word2vec_Medical Specialty Classification6")  #loading the one created above

In [ ]:
word2vec_model.wv.vectors.shape

In [ ]:
#embedding length
embedding_vector_length = word2vec_model.wv.vectors.shape[1]
embedding_vector_length

In [ ]:
# building embedding matrix
#initailiazing matrix to all zeros

embedding_matrix = np.zeros((top_words+1, embedding_vector_length))

tokenizer.word_index.items()

for word,i in sorted(tokenizer.word_index.items(), key = lambda x:x[1]):
  if i > top_words:
    break
  if word in word2vec_model.wv.index_to_key:
    #print(word)
    embedding_vector = word2vec_model.wv[word]
    #print(embedding_vector)
    embedding_matrix[i] = embedding_vector
    #print(embedding_matrix[i])

In [ ]:
# Create an embedding layer
#Output from Embedding is 3 dimension
#batch_size x max_review_length x embedding_vector_length

embedding_layer = Embedding(top_words+1, embedding_vector_length, weights=[embedding_matrix],
                             input_length=padding_max_len,trainable=False)

## Model Training

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,BatchNormalization,Embedding,LSTM ,Bidirectional
from tensorflow.keras import regularizers

In [ ]:
model = Sequential()

model.add(embedding_layer)

model.add(Bidirectional(LSTM(32 , dropout=0.1, recurrent_dropout=0.1)))

model.add(Dense(units=32,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

In [ ]:
model_history = model.fit(X_train, y_train, batch_size=100, epochs=15, validation_data=(X_val, y_val))

In [ ]:
model.save_weights('/content/gdrive/My Drive/lstm4_2.h5')

In [ ]:
from matplotlib import pyplot as plt
# visualise training history
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc="lower right")
plt.show()

In [ ]:
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc="lower right")
plt.show()

## Model Evaluation

In [ ]:
from sklearn import metrics
y_pred = model.predict(X_test)
print(metrics.classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1), target_names=medical_data['medical_specialty'].unique()))